### install dependencies

In [ ]:
!pip install tensorflow

In [ ]:
!pip install matplotlib

In [ ]:
!pip install ibm_watson_machine_learning

In [ ]:
import json

with open('credentials.json') as f:
    data = json.load(f)
    API_KEY = data['API_KEY']
    LOCATION = data['LOCATION']
    PROJECT_ID = data['PROJECT_ID']
    SPACE_ID = data['SPACE_ID']

### create a model

In [ ]:
import tensorflow as tf
from tensorflow.keras.datasets import mnist
import numpy as np
import matplotlib.pyplot as plt
import time

In [ ]:
# load mnist data
(x_train, y_train), (x_test, y_test) = mnist.load_data()
# normalize data
x_train, x_test = x_train / 255.0, x_test / 255.0
# add a channels dimension
x_train = x_train[..., tf.newaxis] 
x_test = x_test[..., tf.newaxis]
# create a dataset
train_ds = tf.data.Dataset.from_tensor_slices(
    (x_train, y_train)).shuffle(10000).batch(32)
test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32)
# create a model
model = tf.keras.models.Sequential([
  tf.keras.layers.Conv2D(32, 3, activation='relu', input_shape=(28, 28, 1)),
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(10)
])
# compile
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [ ]:
# train
model.fit(train_ds, epochs=5)
# evaluate
model.evaluate(test_ds)
# save the model
model.save('mnist_model.h5')

### save the model to watson studio 

In [ ]:
from ibm_watson_machine_learning import APIClient

wml_credentials = {
    "apikey": API_KEY,
    "url": LOCATION
}

wml_client = APIClient(wml_credentials)

# Watson Studio project
wml_client.set.default_project(PROJECT_ID)
# Watson Machine Learning space
wml_client.set.default_space(SPACE_ID)


In [ ]:
# create a tar.gz file with the model
def create_tar_gz_file(model_name):
    import tarfile
    import os
    import shutil
    # create a tar.gz file with the model
    tar = tarfile.open(model_name + '.tar.gz', "w:gz")
    tar.add(model_name + '.h5')
    tar.close()

create_tar_gz_file('mnist_model')
    


In [ ]:
# get the number of parameters in the model
def get_model_parameters(model):
    return np.sum([np.prod(v.get_shape().as_list()) for v in model.trainable_variables])
model_number_of_parameters = get_model_parameters(model)
model_number_of_parameters

In [ ]:
# get the size of the model in megabytes with two decimal places
def get_model_size(model_name):
    import os
    return round(os.path.getsize(model_name + '.tar.gz') / 1000000, 2)
model_size_mb=get_model_size('mnist_model')
str(model_size_mb) + " MB"

In [ ]:
# list of model metadata properties
wml_client.repository.ModelMetaNames.get()

In [ ]:

sofware_spec_uid = wml_client.software_specifications.get_id_by_name("runtime-22.2-py3.10")



metadata = {
            wml_client.repository.ModelMetaNames.NAME: 'mnist_small',
            wml_client.repository.ModelMetaNames.TYPE: 'tensorflow_2.9',
            wml_client.repository.ModelMetaNames.SOFTWARE_SPEC_UID: sofware_spec_uid,
            wml_client.repository.ModelMetaNames.SIZE: {'size_in_mb':str(model_size_mb) + " MB", 'number_of_parameters':str(model_number_of_parameters)}
}

published_model = wml_client.repository.store_model(
    model="mnist_model.tar.gz",
    meta_props=metadata)

In [ ]:
import json

published_model_uid = wml_client.repository.get_model_id(published_model)
model_details = wml_client.repository.get_details(published_model_uid)
print(json.dumps(model_details, indent=2))

In [ ]:
models_details = wml_client.repository.list_models()

### deploy model to watson machine learning

http://ibm-wml-api-pyclient.mybluemix.net/

In [ ]:
metadata = {
    wml_client.deployments.ConfigurationMetaNames.NAME: "Deployment of external Keras model",
    wml_client.deployments.ConfigurationMetaNames.ONLINE: {}
}

created_deployment = wml_client.deployments.create(published_model_uid, meta_props=metadata)

In [ ]:
deployment_uid = wml_client.deployments.get_id(created_deployment)
deployment_uid

In [ ]:
scoring_endpoint = wml_client.deployments.get_scoring_href(created_deployment)
print(scoring_endpoint)

### test deployed model

In [ ]:
# prepare one entry from test_ds for prediction
def prepare_prediction(image_vector, label):
    image_vector = tf.cast(image_vector, tf.float32)
    image_vector = image_vector[tf.newaxis, ...]
    return image_vector, label

In [ ]:
prepared_input=prepare_prediction(x_test[0], y_test[0])
# do prediction locally
results=model.predict(prepared_input[0],prepared_input[1])
# print results
print("predicted digit: {}".format(np.argmax(results)))

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

for i, image in enumerate([x_test[0], x_test[1]]):
    plt.subplot(2, 2, i + 1)
    plt.axis('off')
    plt.imshow(image, cmap=plt.cm.gray_r, interpolation='nearest')

In [ ]:
prepared_input=prepare_prediction(x_test[0], y_test[0])
scoring_payload = {"input_data": [{"values": prepared_input[0].numpy().tolist()}]}
predictions = wml_client.deployments.score(deployment_uid, scoring_payload)

In [ ]:
print("predicted digit: {}".format(np.argmax(predictions["predictions"][0]["values"][0][0])))

In [ ]:
# TODO: include scripts for data prepocessing + beautifying the predictions into a function and deploy it with the model

In [ ]:
# TODO: figure out how to deploy to different enviroments

### do measurements

In [ ]:
# TODO: create multiple models of different sizes and deploy them + measure the performance

In [ ]:
# do a prediction for n images and time it
def predict_n_images(n):
    start = time.time()
    for i in range(n):
        prepared_input=prepare_prediction(x_test[i], y_test[i])
        scoring_payload = {"input_data": [{"values": prepared_input[0].numpy().tolist()}]}
        predictions = wml_client.deployments.score(deployment_uid, scoring_payload)
    end = time.time()
    print("time for {} predictions: {} seconds".format(n, end-start))
    
predictions = predict_n_images(10)